<a href="https://colab.research.google.com/github/djm3622/bayesDL/blob/main/MixOfMultivariate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports/Setup




In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.graph_objects as go
import math

In [2]:
np.random.seed(42)

###Sythetic Data: 2D

Creating 2D sythetic data along with ground truth (mu/sigma/zs).

In [3]:
mu1_2d, mu2_2d, mu3_2d = [2.0, -10.1], [4.3, 6.0], [-1.9, 0.1]
cov1_2d, cov2_2d, cov3_2d = 15.5*np.eye(2), 29.9*np.eye(2), 17.8*np.eye(2)

xs, ys = np.mgrid[-20:20:0.25, -20:20:0.25]
dist1_2d = stats.multivariate_normal(mu1_2d, cov1_2d)
dist2_2d = stats.multivariate_normal(mu2_2d, cov2_2d)
dist3_2d = stats.multivariate_normal(mu3_2d, cov3_2d)

dist1_2d_data = dist1_2d.rvs(1000)
dist2_2d_data = dist2_2d.rvs(1000)
dist3_2d_data = dist3_2d.rvs(1000)

In [4]:
surface1 = go.Surface(
    z=dist1_2d.pdf(np.dstack([xs, ys])),
    showscale=False,
    colorscale='spectral'
)

surface2 = go.Surface(
    z=dist2_2d.pdf(np.dstack([xs, ys])),
    showscale=False,
    colorscale='spectral'
)

surface3 = go.Surface(
    z=dist3_2d.pdf(np.dstack([xs, ys])),
    showscale=False,
    colorscale='spectral'
)

fig = go.Figure(data=[surface1, surface2, surface3])
fig.update_layout(autosize=True)
fig.show()

###Synthetic Data: 3D

Creating 3D sythetic data along with ground truth (mu/sigma/zs).

In [5]:
mu1_3d, mu2_3d, mu3_3d = [2.0, -10.1, 5.0], [4.3, 6.0, -3.3], [-1.9, 0.1, 11.3]
cov1_3d, cov2_3d, cov3_3d = 15.5*np.eye(3), 29.9*np.eye(3), 17.8*np.eye(3)

dist1_3d = stats.multivariate_normal(mu1_3d, cov1_3d)
dist2_3d = stats.multivariate_normal(mu2_3d, cov2_3d)
dist3_3d = stats.multivariate_normal(mu3_3d, cov3_3d)

dist1_3d_data = dist1_3d.rvs(1000)
dist2_3d_data = dist2_3d.rvs(1000)
dist3_3d_data = dist3_3d.rvs(1000)

In [6]:
marker_data1 = go.Scatter3d(
    x=dist1_3d_data[:, 0],
    y=dist1_3d_data[:, 1],
    z=dist1_3d_data[:, 2],
    marker=dict(size=3, color=dist1_3d.pdf(dist1_3d_data), colorscale='spectral'),
    opacity=0.6,
    mode='markers'
)

marker_data2 = go.Scatter3d(
    x=dist2_3d_data[:, 0],
    y=dist2_3d_data[:, 1],
    z=dist2_3d_data[:, 2],
    marker=dict(size=3, color=dist2_3d.pdf(dist2_3d_data), colorscale='algae'),
    opacity=0.6,
    mode='markers'
)

marker_data3 = go.Scatter3d(
    x=dist3_3d_data[:, 0],
    y=dist3_3d_data[:, 1],
    z=dist3_3d_data[:, 2],
    marker=dict(size=3, color=dist3_3d.pdf(dist3_3d_data), colorscale='magma'),
    opacity=0.6,
    mode='markers'
)

fig=go.Figure(data=[marker_data1, marker_data2, marker_data3])
fig.update_layout(autosize=True)
fig.show()

####Step 1: Initialization

Initialize a random value for each ( μₖ , Σₖ , πₖ ).
μₖ is an n by k long random array of floats representing the mean values for the nth dimension of the kth distribution. Σₖ is an n by n by k long random array of floats representing the covariance matrix given n dimensions for the kth distribution. πₖ is k long random floats representing the weight of the kth distribution.

NOTE: We will use n to denote the dimensionality of the data.

NOTE: We will use 𝙸 to represent the size of the data (the amount of elements).

NOTE: We will use k to denote ith of distributions given K distributions.

In [19]:
def init(dim, ks):
  mus = []
  covs = []
  pis = []

  for _ in range(ks):
    mus.append(40 * np.random.rand(dim) - 20)
    covs.append(20 * np.random.rand(1) * np.eye(dim))
    pis.append(1/ks)

  return mus, covs, pis

#### Step 2: Expectation

Calculate the probability that the data point belongs to cluster (k) using the below equation. And update its respective measure. Below, 𝚚ⁱₖ is the measure for the given data point and the given distribution. qₖ is the weight of the distribution. μₖ and Σₖ are distribution parameters. m is ith distribution of the set of all distributions K. Zᵢ corresponds to a RV that is the mixing coeffiencts of each distribution.

\begin{align}
𝚚ⁱₖ = Pr(z_i = k| x_i) \overset{\text{by Bayes' rule}}{\rightarrow} \frac{πₖ · Pr(x_i|z_i = k)}{\sum_{m=1}^{K} πₘ · Pr(x_i|z_i = m)} = \frac{πₖ · N(𝑥ᵢ |μₖ, Σₖ)}{\sum_{m=1}^{K} πₘ · N(𝑥ᵢ |μₘ, Σₘ)}
\end{align}

Given that,

\begin{align}
N(𝑥ᵢ |μₖ, Σₖ) = \frac{1}{(2\pi)^{\frac{n}{2}}|\Sigma_k|^{\frac{1}{2}}} \exp\left(-\frac{1}{2} (x_i - \mu_k)^T \Sigma_k^{-1} (x_i - \mu_k)\right)
\end{align}

Therefore,

\begin{align}
𝚚ⁱₖ = \frac{πₖ · \frac{1}{(2\pi)^{\frac{n}{2}}|\Sigma_k|^{\frac{1}{2}}} \exp\left(-\frac{1}{2} (x_i - \mu_k)^T \Sigma_k^{-1} (x_i - \mu_k)\right)}{\sum_{m=1}^{k} πₘ · \frac{1}{(2\pi)^{\frac{n}{2}}|\Sigma_m|^{\frac{1}{2}}} \exp\left(-\frac{1}{2} (x_i - \mu_m)^T \Sigma_m^{-1} (x_i - \mu_m)\right)}
\end{align}

Returned will be an n by 𝙸 array of floats descibing the measure of the kth cluster on ith element.

In [ ]:
def normal_pdf(mu, sigma, dim, xi):
  return (1/(((2*math.pi)**(dim/2)) * (np.linalg.norm(sigma)**(1/2)))) * np.exp( (-1/2) * ( (xi - mu).T @ np.linalg.inv(sigma) @ (xi - mu)))

def expectation(mus, covs, pis, data, dim):
  qik = [[0 for _ in range(len(data))] for _ in range(len(pis))]

  for i in range(len(data)):
    for k in range(len(pis)):
      for m in range(len(pis)):
        denominator += pis[m] * normal_pdf(mus[m], covs[m], dim, data[i])
      numerator = pis[k] * normal_pdf(mus[k], covs[k], dim, data[i])
      qik[k][i] = numerator / denominator

  return qik

#### Step 3: Maximization

\begin{align}
\mathscr{L} = \sum_{i=1}^{I} \sum_{k=1}^{K} 𝚚ⁱₖ · log\left(\frac{Pr(x_i , z_i = k)}{𝚚ⁱₖ}\right)
\end{align}

\begin{align}
\ = \sum_{i=1}^{I} \sum_{k=1}^{K} 𝚚ⁱₖ · log(π_k · N(x_i | μ_k, Σ_k)) - \sum_{i=1}^{I} \sum_{k=1}^{K} 𝚚ⁱₖ · log(𝚚ⁱₖ)
\end{align}

Given 𝚚ⁱₖ is a constant measure and N(𝑥ᵢ |μₖ, Σₖ) is defined above,

\begin{align}
\ = \sum_{i=1}^{I} \sum_{k=1}^{K} 𝚚ⁱₖ · log\left(π_k · \frac{1}{(2\pi)^{\frac{n}{2}}|\Sigma_k|^{\frac{1}{2}}} \exp\left(-\frac{1}{2} (x_i - \mu_k)^T \Sigma_k^{-1} (x_i - \mu_k)\right)\right) - \sum_{i=1}^{I} \sum_{k=1}^{K} 𝚚ⁱₖ · log(𝚚ⁱₖ)
\end{align}

\begin{align}
\ = \sum_{i=1}^{I} \sum_{k=1}^{K} 𝚚ⁱₖ \left[ log\left(π_k\right) - \frac{d}{2}log\left(2π\right) + log\left(\left| Σ_k^{-1}\right|\right) - \frac{1}{2}\left( x_i - μ_k\right)^TΣ_k^{-1}\left( x_i - μ_k\right)\right] \rightarrow \underset{μ_k , Σ_k , π_k}{\text{Maximize}}
\end{align}

In respect to πₖ,

\begin{align}
\frac{δ}{δπ_k} = \sum_{i=1}^{I} \sum_{k=1}^{K} 𝚚ⁱₖ · \frac{1}{π_k} \overset{max}{\rightarrow} \sum_{i=1}^{I} \sum_{k=1}^{K} 𝚚ⁱₖ · \frac{1}{π_k} = 0
\end{align}

In respect to μₖ,

\begin{align}
\frac{δ}{δμ_k} = \sum_{i=1}^{I} \sum_{k=1}^{K} 𝚚ⁱₖ · Σ_k^{-1}\left( x_i - μ_k\right) \overset{max}{\rightarrow} \sum_{i=1}^{I} \sum_{k=1}^{K} 𝚚ⁱₖ · Σ_k^{-1}x_i - 𝚚ⁱₖ · Σ_k^{-1}μ_k = 0
\end{align}

\begin{align}
\sum_{i=1}^{I} \sum_{k=1}^{K} 𝚚ⁱₖ · Σ_k^{-1}x_i = \sum_{i=1}^{I}\sum_{k=1}^{K} 𝚚ⁱₖ · Σ_k^{-1}μ_k
\end{align}

In respect to Σₖ,

\begin{align}
\frac{δ}{δΣ_k^{-1}} =
\end{align}



In [ ]:
def maximation():
  pass

#### Training

Loop until convergence. Convergce will be decided based on the less than comparison of a tolorance variable and difference in (METRIC TBD).

LOOK INTO PLOTTY ANIMATION TO FORM THE RANDOM INIT TO GROUND TRUTH